<a href="https://colab.research.google.com/github/eljustus/Automated-Segmentation-Campaign/blob/main/Segmentation_Campaign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
import os


In [2]:
conn = sqlite3.connect('campaign.db')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SMS_Campaign.csv to SMS_Campaign.csv


In [ ]:
df = pd.read_csv("SMS_Campaign.csv")

In [ ]:
df.head()

,S/N,First_Name,Phone_Number,DOB,Age
0,1.0,alex,4.460063e+11,10-Sep-76,49.0
1,2.0,alexander,4.424710e+11,20-Sep-86,39.0
2,3.0,alfred,4.478642e+11,20-Jul-68,57.0
3,4.0,ali,4.467231e+11,04-Feb-91,34.0
4,5.0,alice,4.490676e+11,15-Sep-03,22.0


Clean Table

In [ ]:
df.columns = df.columns.str.strip().str.lower()

In [ ]:
df.head()

,s/n,first_name,phone_number,dob,age
0,1.0,alex,4.460063e+11,10-Sep-76,49.0
1,2.0,alexander,4.424710e+11,20-Sep-86,39.0
2,3.0,alfred,4.478642e+11,20-Jul-68,57.0
3,4.0,ali,4.467231e+11,04-Feb-91,34.0
4,5.0,alice,4.490676e+11,15-Sep-03,22.0


In [ ]:
required_cols = ['age', 'phone_number']  # Change if your columns are e.g., 'Age', 'Phone'
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"Missing columns! Found: {df.columns.tolist()}")

In [ ]:
# Load into SQLite (replaces if exists)
df.to_sql('customers', conn, if_exists='replace', index=False)

print("Dataset loaded successfully!")
print(f"Rows: {len(df)}, Columns: {df.columns.tolist()}")
print(df.head())  # Preview

Dataset loaded successfully!
Rows: 222, Columns: ['s/n', 'first_name', 'phone_number', 'dob', 'age']
   s/n first_name  phone_number        dob   age
0  1.0       alex  4.460063e+11  10-Sep-76  49.0
1  2.0  alexander  4.424710e+11  20-Sep-86  39.0
2  3.0     alfred  4.478642e+11  20-Jul-68  57.0
3  4.0        ali  4.467231e+11  04-Feb-91  34.0
4  5.0      alice  4.490676e+11  15-Sep-03  22.0


In [ ]:
segments = {
    1: {"age_min": 18, "age_max": 28, "message": "Buy one bottle of coke and get one free"},
    2: {"age_min": 29, "age_max": 39, "message": "Buy one Bottle of coke and get one bottle of Fanta free"},
    3: {"age_min": 40, "age_max": 50, "message": "Buy a bottle of Coke and get a bottle of water free"},
    4: {"age_min": 51, "age_max": 999, "message": "Buy one bottle water and get one bottle water free"}
}

In [ ]:
df = pd.read_sql_query("SELECT * FROM customers WHERE age >= 18", conn)

In [ ]:
# Assign segment
def get_segment(age):
    for seg, info in segments.items():
        if info['age_min'] <= age <= info['age_max']:
            return seg
    return None

df['segment'] = df['age'].apply(get_segment)
df['message'] = df['segment'].map({k: v['message'] for k, v in segments.items()})

In [ ]:
# Group phones by segment
grouped = df.groupby('segment').agg({
    'phone_number': list,
    'message': 'first'
}).reset_index()

In [ ]:
##Step 5: "Send" messages (simulate print; for real, use Twilio)
for _, row in grouped.iterrows():
    segment = row['segment']
    message = row['message']
    phones = row['phone_number']

    print(f"\n--- Segment {segment} ---")
    print(f"Message: {message}")
    print("Sending to phones:", phones)

    # For real SMS (uncomment after pip install twilio and setup)
    # from twilio.rest import Client
    # client = Client('YOUR_TWILIO_SID', 'YOUR_TWILIO_AUTH_TOKEN')
    # for phone in phones:
    #     client.messages.create(body=message, from_='YOUR_TWILIO_NUMBER', to=phone)
    #     print(f"Sent to {phone}")

conn.close()


--- Segment 1 ---
Message: Buy one bottle of coke and get one free
Sending to phones: [449067600135.0, 449716042304.0, 444189826344.0, 447695850036.0, 443674954943.0, 447842693293.0, 443359896032.0, 448085499460.0, 447838067240.0, 443905448294.0, 445299942752.0, 444343297612.0, 446161456283.0, 449485389533.0, 449855790128.0, 445919118012.0, 441151421269.0, 448057186508.0, 441720468188.0, 442513329100.0, 444491907839.0, 446918562740.0]

--- Segment 2 ---
Message: Buy one Bottle of coke and get one bottle of Fanta free
Sending to phones: [442471008074.0, 446723057358.0, 446352116454.0, 444924932897.0, 445146375169.0, 449211519295.0, 446198185207.0, 443535727919.0, 447391173692.0, 446308921662.0, 443146928647.0, 443819157015.0, 448616490772.0, 442933190999.0, 444811652677.0, 444799780687.0, 442005342400.0, 441858854028.0, 444764089892.0, 442654142118.0, 447022047469.0, 441147186270.0, 449976592040.0, 448630794369.0, 445093122236.0, 449735502135.0, 448559259604.0, 448187670431.0, 44688452